In [46]:
import os 

In [74]:

%pwd

'/home/user'

In [75]:
os.chdir('/home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project')

In [76]:
%pwd

'/home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project'

In [83]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [84]:
from kidneyDisease.constants import *
from kidneyDisease.utils.common import read_yaml,create_directories
import tensorflow as tf 


In [88]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [89]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


In [93]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    

In [94]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-08-13 20:33:54,994: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-13 20:33:55,006: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-13 20:33:55,010: INFO: common: created directory at: artifacts]


[2024-08-13 20:33:55,015: INFO: common: created directory at: artificats/training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/7


2024-08-13 20:33:56.318059: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


23/23 [==============================] - ETA: 0s - loss: 12.7411 - accuracy: 0.5674

2024-08-13 20:34:30.067499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


23/23 [==============================] - 41s 2s/step - loss: 12.7411 - accuracy: 0.5674 - val_loss: 21.7701 - val_accuracy: 0.4750
Epoch 2/7
23/23 [==============================] - 39s 2s/step - loss: 6.6248 - accuracy: 0.6657 - val_loss: 21.8939 - val_accuracy: 0.4750
Epoch 3/7
23/23 [==============================] - 39s 2s/step - loss: 3.4371 - accuracy: 0.7865 - val_loss: 7.8438 - val_accuracy: 0.4375
Epoch 4/7
23/23 [==============================] - 38s 2s/step - loss: 3.6689 - accuracy: 0.7697 - val_loss: 17.1916 - val_accuracy: 0.4000
Epoch 5/7
23/23 [==============================] - 38s 2s/step - loss: 4.8281 - accuracy: 0.7416 - val_loss: 2.0954 - val_accuracy: 0.7375
Epoch 6/7
23/23 [==============================] - 38s 2s/step - loss: 1.7205 - accuracy: 0.8371 - val_loss: 0.4319 - val_accuracy: 0.8500
Epoch 7/7
23/23 [==============================] - 39s 2s/step - loss: 1.2591 - accuracy: 0.8736 - val_loss: 15.6224 - val_accuracy: 0.4000
